* You can use this notebook to load your best model weights and test at test sets.
* And ensure that you disabled the internet so you can submit your result.
* Add your model weights though the top right corner button.


In [ ]:
import sys
package_path = "../input/pytorch-img-class-models/pytorch-image-models-master"
sys.path.append(package_path)

import timm

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GroupKFold
import pandas as pd
import numpy as np
import os
import cv2
import time
import datetime
import warnings
import gc
# import albumentations as A
# from albumentations.pytorch import ToTensorV2
from PIL import Image

In [ ]:
warnings.simplefilter('ignore')
torch.manual_seed(47)
np.random.seed(47)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class Dataset(Dataset):
    def __init__(self, df: pd.DataFrame, imfolder: str, train: bool = True, transforms = None,):
        self.df = df
        self.imfolder = imfolder
        self.transforms = transforms
        self.train = train


    def __getitem__(self, index):
        im_path = os.path.join(self.imfolder, self.df.iloc[index]['image_id'])
#         x = cv2.imread(im_path)
#         x = cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        x = Image.open(im_path) #for torchvision.transform
        

        if self.transforms:
            x = self.transforms(x)   #FOR torchvision.transform
#             augmented = self.transforms(image=x)
#             x = augmented['image']

        if self.train:
            y = self.df.iloc[index]['label']
            return x, y
        else:
            return x

    def __len__(self):
        return len(self.df)

In [ ]:
class CustomEffNet(nn.Module):
    def __init__(self, model_name:str="tf_efficientnet_b3", pretrained:bool=True, n_out:int=5):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)

#         n_features = self.model.classifier.in_features   #efficientnet
        n_features = self.model.fc.in_features    #resnet
        self.model.fc = nn.Linear(n_features, n_out)

    def forward(self, x):
        x = self.model(x)
        return x
model = CustomEffNet("resnext50_32x4d", False, 5)

In [ ]:
# print(model)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=512, scale=(0.8, 0.8)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=32. / 255.,saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.CenterCrop(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
# train_transform = A.Compose([
# #     A.Resize(380,380),
# #     A.CenterCrop(240,240,p=1),
#     A.Resize(256,256),
# #     A.InvertImg(p=0.6),
#     A.RandomShadow(shadow_roi=(0, 0.5, 1, 1), num_shadows_lower=1, num_shadows_upper=4, shadow_dimension=5, always_apply=False),
#     A.Flip(p=0.75),
#     A.RandomRotate90(p=0.75),
#     A.ColorJitter(brightness=0.4,contrast=0.5,saturation=0.5,hue=0.3,p=0.7,always_apply=False),
# #     A.ToTensor(),
#     A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
#     ToTensorV2(p=1.0)
# ])
# test_transform = A.Compose([
# #     A.Resize(380,380),
#     A.CenterCrop(240,240,p=1),
# #     A.Resize(256,256),
# #     A.ToTensor(),
#     A.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
#     ToTensorV2(p=1.0)
# ])

In [ ]:
skf = GroupKFold(n_splits=3)


train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
test_df = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')


b_df = train_df.loc[train_df['label']==3].sample(frac = 0.4,replace = False)
sample_df = pd.concat([b_df,train_df.loc[train_df['label']!=3]])


test = Dataset(df=test_df,
                       imfolder='../input/cassava-leaf-disease-classification/test_images',
                       train=False,
                       transforms=test_transform,
                       )

In [ ]:
epochs = 50  
es_patience = 3  
TTA = 1 


test_loader = DataLoader(dataset=test, batch_size=16, shuffle=False, num_workers=2)

In [ ]:
if __name__ == '__main__':

    z_pred = torch.zeros((len(test), 5), dtype=torch.float32, device=device)
    for num in range(0,2):
        print('=' * 20, 'Fold', num, '=' * 20)
        model_path = f'../input/efficient/0122_Resnet_{num}.pth'
        best_val = None  # Best validation score within this fold
        patience = es_patience  # Current patience counter
#         model = torch.load(model_path)  # Loading best model of this fold
        model = torch.load(model_path)
        model = model.to(device)
        model.eval()  # switch model to the evaluation mode
        with torch.no_grad():
            start_time = time.time()
            for _ in range(TTA):
                for i, x_test in enumerate(test_loader):
                    x_test = torch.tensor(x_test, device=device, dtype=torch.float32)
                    z_test = model(x_test)
                    z_pre = F.softmax(z_test)
                    z_pred[i*2:i*2+x_test.size(0)] += z_pre
            z_pred /= TTA
            print('Test time: {}'.format(str(datetime.timedelta(seconds=time.time() - start_time))[:7]))
        gc.collect()

    z_pred /= skf.n_splits
    z_preds = torch.argmax(z_pred,dim=1)
    sub = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
    sub['label'] = z_preds.cpu().numpy().reshape(-1, )
    sub.to_csv('submission.csv', index=False)